# Chicago Bike Infrastructure Project
<h3>Capstone Project for Data Analytics Certificate<br>
University of Texas<br><br>
Samantha Goodman
<br>December 2021</h3><br>
This notebook is part 2 of a 5 part series.<br>
       1 - Bike Shops from FourSquare API<br>
    <b>2 - Bike Infrastructure<br></b>
        3 - Background information about neighborhoods<br>
        4 - Analysis<br>
        5 - Model Building and Predictions<br><br>
    Questions this project aims to answer:<br>
    <ul><li>Which community areas (neighborhoods) have the most bike infrastructure, and which have the least?</li>
<li>Are there areas that show an unmet demand for bike infrastructure (higher rates of Divvy trips, but lower rates of bike lanes and repair shops)?</li>
<li>Can I predict bike infrastructure levels based on demographic or community health data?


In [2]:
# Import Libraries
import numpy as np
import pandas as pd

First import list of Chicago neighborhoods - referred to as Community Areas

In [3]:
# Import CSV to dataframe
# Source = City of Chicago data portal
neighborhoods = pd.read_csv('Chicago_community_areas.csv')
# Make all column names lowercase
neighborhoods.columns= neighborhoods.columns.str.lower()

In [4]:
neighborhoods.shape

(77, 3)

There are 77 official neighborhoods in Chicago. We will keep this number in mind when chekcing our datasets going forward.

# Next Bike Shops for each neighborhood

In [5]:
# Import CSV to dataframe
# Source = FourSquare, see 1-BikeShopsFromFourSquare notebook
bike_shops = pd.read_csv('bike_shops_comm.csv', index_col=0)
# Make all column names lowercase
bike_shops.columns= bike_shops.columns.str.lower()

In [6]:
bike_shops.head()

,address,categories,city,lat,lng,name,community,area_num_1
field_1,,,,,,,,
0,2130 N Milwaukee Ave,Bike Shop,Chicago,41.919867,-87.692609,The Bike Lane,LOGAN SQUARE,22
1,2769 N Milwaukee Ave,Bike Shop,Chicago,41.931570,-87.711180,Boulevard Bikes,LOGAN SQUARE,22
2,2056 N Damen Ave,Bike Shop,Chicago,41.919360,-87.677780,BFF Bikes,LOGAN SQUARE,22
3,3031 W Armitage Ave,Bike Shop,Chicago,41.917484,-87.703692,Smart Bike Parts Inc,LOGAN SQUARE,22
4,2959 N Lincoln Avenue,Coffee Shop,Chicago,41.935760,-87.662830,Heritage Bicycles,LAKE VIEW,6


In [7]:
bike_shops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 143
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   address     102 non-null    object 
 1   categories  106 non-null    object 
 2   city        106 non-null    object 
 3   lat         106 non-null    float64
 4   lng         106 non-null    float64
 5   name        106 non-null    object 
 6   community   106 non-null    object 
 7   area_num_1  106 non-null    int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 7.5+ KB


In [8]:
# Create a dataframe of grouped data by with count of bike shops by neighborhood
bike_shops_grouped = pd.DataFrame(bike_shops.groupby('community')['name'].count().reset_index()).copy()
bike_shops_grouped.head()

,community,name
0,ALBANY PARK,3
1,AVONDALE,1
2,BEVERLY,2
3,BRIDGEPORT,2
4,DOUGLAS,1


In [9]:
# Rename column to reflect that it's the number of shops
bike_shops_grouped.columns = bike_shops_grouped.columns.str.replace('name', 'num_bike_shops')

# Sort descending by num of bike shops, just to see
bike_shops_grouped.sort_values(by=['num_bike_shops'], inplace=True, ascending=False)

bike_shops_grouped.head()

,community,num_bike_shops
31,WEST TOWN,11
15,LOGAN SQUARE,11
12,LAKE VIEW,10
13,LINCOLN PARK,8
20,NEAR NORTH SIDE,6


Add bike shops to neighborhood database and rename as bike_infrastructure

In [10]:
# Join neighborhoods and bike shop dataframes on the 'community' column
bike_infrastructure = neighborhoods.merge(bike_shops_grouped, how='left', on='community')

In [11]:
bike_infrastructure.tail()

,comm_num,community,area_kmsq,num_bike_shops
72,73,WASHINGTON HEIGHTS,79.635753,NaN
73,74,MOUNT GREENWOOD,75.584290,1.0
74,75,MORGAN PARK,91.877341,NaN
75,76,OHARE,371.835608,NaN
76,77,EDGEWATER,48.449991,3.0


In [12]:
bike_infrastructure.shape

(77, 4)

# Next up, add in number of Divvy Stations by Neighborhood

In [13]:
# Import CSV to dataframe
# Source = City of Chicago Data Portal, with community area added
divvy_stations = pd.read_csv('divvy_stations_comm.csv')
# Make all column names lowercase
divvy_stations.columns= divvy_stations.columns.str.lower()

In [14]:
divvy_stations.head()

,id,station name,total docks,docks in service,status,latitude,longitude,location,area_numbe,community
0,373,Kedzie Ave & Chicago Ave,11,11,In Service,41.895465,-87.706128,"(41.89546530305, -87.7061278837)",23,HUMBOLDT PARK
1,102,Stony Island Ave & 67th St,11,11,In Service,41.773458,-87.585340,"(41.77345849948, -87.5853397391)",42,WOODLAWN
2,438,Central Park Ave & Ogden Ave,15,15,In Service,41.854064,-87.715117,"(41.854063503874464, -87.71511733531952)",29,NORTH LAWNDALE
3,560,Marshfield Ave & 59th St,11,11,In Service,41.786833,-87.666215,"(41.786833, -87.666215)",67,WEST ENGLEWOOD
4,290,Kedzie Ave & Palmer Ct,15,15,In Service,41.921525,-87.707322,"(41.921525, -87.707322)",22,LOGAN SQUARE


In [15]:
# Create a dataframe of grouped data by with count of divvy stations by neighborhood
divvy_stations_grouped = pd.DataFrame(divvy_stations.groupby('community')['id'].count().reset_index()).copy()
divvy_stations_grouped.head

<bound method NDFrame.head of          community  id
0      ALBANY PARK   6
1   ARCHER HEIGHTS   5
2    ARMOUR SQUARE   8
3   AUBURN GRESHAM   5
4           AUSTIN  27
..             ...  ..
64       WEST LAWN   4
65    WEST PULLMAN   4
66      WEST RIDGE  14
67       WEST TOWN  40
68        WOODLAWN  11

[69 rows x 2 columns]>

In [16]:
# Rename column to reflect that it's the number of stations
divvy_stations_grouped.columns = divvy_stations_grouped.columns.str.replace('id', 'num_divvy_stations')

# Sort descending by num of bike shops
divvy_stations_grouped.sort_values(by=['num_divvy_stations'], inplace=True, ascending=False)

In [17]:
divvy_stations_grouped.head()

,community,num_divvy_stations
44,NEAR WEST SIDE,59
42,NEAR NORTH SIDE,52
67,WEST TOWN,40
36,LOOP,40
33,LINCOLN PARK,39


In [18]:
# Join divvy stations to bike infrastructure dataframe
bike_infrastructure = bike_infrastructure.merge(divvy_stations_grouped, how='left', on='community')

In [19]:
bike_infrastructure.shape

(77, 5)

In [20]:
bike_infrastructure.head()

,comm_num,community,area_kmsq,num_bike_shops,num_divvy_stations
0,1,ROGERS PARK,51.259902,3.0,17.0
1,2,WEST RIDGE,98.429095,2.0,14.0
2,3,UPTOWN,65.095643,3.0,14.0
3,4,LINCOLN SQUARE,71.352328,3.0,18.0
4,5,NORTH CENTER,57.054168,4.0,14.0


# Add (historical) bike rack locations

In [21]:
# Import CSV to dataframe
# Source = City of Chicago Data Portal, with community area added
bike_racks = pd.read_csv('Bike_Racks_comm.csv')
# Make all column names lowercase
bike_racks.columns= bike_racks.columns.str.lower()

In [22]:
bike_racks.head()

,rackid,address,ward,community area,community name,totinstall,latitude,longitude,historical,f12,f13,location
0,3492,1391 N Milwaukee Ave,1,24,West Town,1,41.907164,-87.672159,1,41.907164,-87.672159,"(41.907164, -87.672159)"
1,1284,1554 E 55th St,4,41,Hyde Park,1,41.795283,-87.587632,1,41.795283,-87.587632,"(41.795283, -87.587632)"
2,5945,1001 N Damen Ave,1,24,West Town,1,41.899661,-87.677036,1,41.899661,-87.677036,"(41.899661, -87.677036)"
3,3241,1932 W Lawrence Ave,47,4,Lincoln Square,2,41.968910,-87.677444,1,41.968910,-87.677444,"(41.96891, -87.677444)"
4,1349,2157 W 19th St,25,31,Lower West Side,1,41.855659,-87.679748,1,41.855659,-87.679748,"(41.855659, -87.679748)"


In [23]:
# Create a dataframe of grouped data by with count of divvy stations by neighborhood
bike_racks_grouped = pd.DataFrame(bike_racks.groupby('community area')['rackid'].count().reset_index()).copy()
bike_racks_grouped

,community area,rackid
0,1,119
1,2,51
2,3,173
3,4,146
4,5,137
...,...,...
72,73,29
73,74,25
74,75,43
75,76,5


In [24]:
# Join bike to bike infrastructure dataframe
bike_infrastructure = bike_infrastructure.merge(bike_racks_grouped, how='left', right_on='community area', left_on='comm_num')

In [25]:
bike_infrastructure.head()

,comm_num,community,area_kmsq,num_bike_shops,num_divvy_stations,community area,rackid
0,1,ROGERS PARK,51.259902,3.0,17.0,1,119
1,2,WEST RIDGE,98.429095,2.0,14.0,2,51
2,3,UPTOWN,65.095643,3.0,14.0,3,173
3,4,LINCOLN SQUARE,71.352328,3.0,18.0,4,146
4,5,NORTH CENTER,57.054168,4.0,14.0,5,137


In [26]:
# Rename column to reflect that it's the number of racks
bike_infrastructure.columns = bike_infrastructure.columns.str.replace('rackid', 'num_bike_racks')

# Drop extraneous community area column
bike_infrastructure.drop('community area', axis=1, inplace=True)

# Add bike lane info by community area

In [27]:
# Import CSV to dataframe
# Source = City of Chicago Data Portal
bike_lanes = pd.read_csv('bike_lanes_length.csv')
# Make all column names lowercase
bike_lanes.columns= bike_lanes.columns.str.lower()

In [28]:
bike_lanes.head()

,r_f_addr,mi_ctrline,oneway_dir,displayrou,r_t_addr,f_street,objectid,pre_dir,st_type,t_street,...,st_name,area,perimeter,comarea_,comarea_id,area_numbe,community,shape_area,shape_len,length
0,3200,0.495170,NaN,BIKE LANE,3556,S CENTRAL PARK AVE,1,W,RD,S KEDZIE AVE,...,MARQUETTE,0,0,0,0,66,CHICAGO LAWN,9.827947e+07,40073.099838,796.893431
1,3158,0.252403,NaN,SHARED-LANE,3000,S KEDZIE AVE,2,W,RD,S SACRAMENTO AVE,...,MARQUETTE,0,0,0,0,66,CHICAGO LAWN,9.827947e+07,40073.099838,406.200476
2,1900,0.757321,NaN,BIKE LANE,1400,S DAMEN AVE,3,W,ST,S LOOMIS ST,...,18TH,0,0,0,0,31,LOWER WEST SIDE,8.155072e+07,43229.372704,1218.776676
3,8101,0.754309,NaN,BUFFERED BIKE LANE,8701,E 81ST ST,4,S,DR,E 87TH ST,...,LAKE SHORE,0,0,0,0,46,SOUTH CHICAGO,9.327219e+07,52435.979431,1213.910493
4,3100,1.522768,NaN,PROTECTED BIKE LANE,2060,N KEDZIE AVE,5,W,ST,N DAMEN AVE,...,LAKE,0,0,0,0,27,EAST GARFIELD PARK,5.388322e+07,31514.625957,1216.525780


In [29]:
# value counts of each type of bike lane
print(bike_lanes['displayrou'].value_counts())

BIKE LANE                362
BUFFERED BIKE LANE       318
SHARED-LANE              155
PROTECTED BIKE LANE       96
OFF-STREET TRAIL          82
NEIGHBORHOOD GREENWAY     60
ACCESS PATH               37
Name: displayrou, dtype: int64


In [28]:
# Create a dataframe of grouped data by with length of bike lanes by neighborhood
bike_lanes_grouped= pd.DataFrame(bike_lanes.groupby('community')['length'].sum().reset_index()).copy()
bike_lanes_grouped

,community,length
0,ALBANY PARK,7732.467790
1,ARMOUR SQUARE,2976.688654
2,ASHBURN,1706.684185
3,AUBURN GRESHAM,12457.547086
4,AUSTIN,10417.968135
...,...,...
66,WEST LAWN,37.747682
67,WEST PULLMAN,7989.227428
68,WEST RIDGE,8700.467641
69,WEST TOWN,30465.472288


In [29]:
# Sort descending by length of bike lanes, just to see
bike_lanes_grouped.sort_values(by=['length'], ascending=False)

,community,length
45,NEAR WEST SIDE,42912.734734
69,WEST TOWN,30465.472288
35,LAKE VIEW,20526.927358
36,LINCOLN PARK,19619.909810
43,NEAR NORTH SIDE,18132.435682
...,...,...
20,EDISON PARK,594.685875
23,FULLER PARK,568.667373
10,BURNSIDE,108.119621
66,WEST LAWN,37.747682


In [30]:
# Join bike lanes to bike infrastructure dataframe
bike_infrastructure = bike_infrastructure.merge(bike_lanes_grouped, how='left', on='community')
bike_infrastructure.head()

,comm_num,community,area_kmsq,num_bike_shops,num_divvy_stations,num_bike_racks,length
0,1,ROGERS PARK,51.259902,3.0,17.0,119,11935.328336
1,2,WEST RIDGE,98.429095,2.0,14.0,51,8700.467641
2,3,UPTOWN,65.095643,3.0,14.0,173,16894.242231
3,4,LINCOLN SQUARE,71.352328,3.0,18.0,146,9494.829532
4,5,NORTH CENTER,57.054168,4.0,14.0,137,11554.882769


In [31]:
# Rename column to reflect that it's the number of racks
bike_infrastructure.columns = bike_infrastructure.columns.str.replace('length', 'length_bike_lanes_m')

# Final bike related dataset to add is Divvy Trips (June 2019)

In [32]:
# Import CSV to dataframe
# Source = City of Chicago Data Portal, with community area added
divvy_trips = pd.read_csv('divvy_trips_June2019_comm.csv')
# Make all column names lowercase
divvy_trips.columns= divvy_trips.columns.str.lower()

In [33]:
divvy_trips.head()

,trip id,from station id,from station name,area_numbe,community
0,22923612,66,Clinton St & Lake St,28,NEAR WEST SIDE
1,22923613,25,Michigan Ave & Pearson St,8,NEAR NORTH SIDE
2,22923614,239,Western Ave & Leland Ave,4,LINCOLN SQUARE
3,22923615,251,Clarendon Ave & Leland Ave,3,UPTOWN
4,22923616,29,Noble St & Milwaukee Ave,24,WEST TOWN


In [34]:
divvy_trips.shape

(472754, 5)

472,000 trips were made in the month of June 2019. I've chosen to use this sample because the complete dataset was too large to download. I've also only selected the 'from' data, and added the community area that the user started from.

In [35]:
# Create a dataframe of grouped data by with number of divvy trips originating in each neighborhood
divvy_trips_grouped= pd.DataFrame(divvy_trips.groupby('community')['trip id'].count().reset_index()).copy()
divvy_trips_grouped.head()

,community,trip id
0,ALBANY PARK,616
1,ARMOUR SQUARE,1805
2,AUSTIN,222
3,AVALON PARK,23
4,AVONDALE,1036


In [36]:
# Rename column to reflect that it's the number of trips
divvy_trips_grouped.columns = divvy_trips_grouped.columns.str.replace('trip id', 'num_divvy_trips')

In [37]:
# Join bike lanes to bike infrastructure dataframe
bike_infrastructure = bike_infrastructure.merge(divvy_trips_grouped, how='left', on='community')
bike_infrastructure.head()

,comm_num,community,area_kmsq,num_bike_shops,num_divvy_stations,num_bike_racks,length_bike_lanes_m,num_divvy_trips
0,1,ROGERS PARK,51.259902,3.0,17.0,119,11935.328336,3174.0
1,2,WEST RIDGE,98.429095,2.0,14.0,51,8700.467641,695.0
2,3,UPTOWN,65.095643,3.0,14.0,173,16894.242231,11523.0
3,4,LINCOLN SQUARE,71.352328,3.0,18.0,146,9494.829532,6247.0
4,5,NORTH CENTER,57.054168,4.0,14.0,137,11554.882769,4659.0


Next a little data cleaning to ensure no n/a or incorrect types

In [38]:
# Fill in n/a values with 0
bike_infrastructure = bike_infrastructure.fillna(0)

# Recast floats as ints
bike_infrastructure['num_divvy_stations'] = bike_infrastructure['num_divvy_stations'].astype(int)
bike_infrastructure['num_bike_shops'] = bike_infrastructure['num_bike_shops'].astype(int)
bike_infrastructure['num_divvy_trips'] = bike_infrastructure['num_divvy_trips'].astype(int)

In [39]:
bike_infrastructure.head()

,comm_num,community,area_kmsq,num_bike_shops,num_divvy_stations,num_bike_racks,length_bike_lanes_m,num_divvy_trips
0,1,ROGERS PARK,51.259902,3,17,119,11935.328336,3174
1,2,WEST RIDGE,98.429095,2,14,51,8700.467641,695
2,3,UPTOWN,65.095643,3,14,173,16894.242231,11523
3,4,LINCOLN SQUARE,71.352328,3,18,146,9494.829532,6247
4,5,NORTH CENTER,57.054168,4,14,137,11554.882769,4659


In [41]:
bike_infrastructure.shape

(77, 8)

In [40]:
# Save dataframe to CSV for use in analysis notebook
bike_infrastructure.to_csv('bike_infrastructure_by_comm.csv')